In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import KFold
from IPython.display import HTML, display
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
train = pd.read_csv("/kaggle/input/home-data-for-ml-course/train.csv")
test = pd.read_csv("/kaggle/input/home-data-for-ml-course/test.csv")
sample_submission = pd.read_csv("/kaggle/input/home-data-for-ml-course/sample_submission.csv")

In [ ]:
train.head()

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
train.columns

In [ ]:
test.columns

In [ ]:
#オブジェクトのカラムの名前と数値のカラムの名前をそれぞれ抽出

quantitative = [f for f in train.columns if train.dtypes[f] != 'object']
quantitative.remove('SalePrice')
quantitative.remove('Id')
qualitative = [f for f in train.columns if train.dtypes[f] == 'object']

In [ ]:
print(len(qualitative))

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:
(train.isnull().sum().sort_values(ascending = False)/len(train)).head(20)

In [ ]:
train.info()

In [ ]:
train["SalePrice"].describe()

In [ ]:
train.describe().columns

In [ ]:
a = train[['Id', 'MSSubClass', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold', 'SalePrice']].corr()

In [ ]:
a[a["SalePrice"] >= 0.4]["SalePrice"]

In [ ]:
#ターゲットと相関が高いものたちどうしの相関関係
train[['OverallQual','YearBuilt','YearRemodAdd','MasVnrArea','TotalBsmtSF','1stFlrSF','GrLivArea','FullBath','TotRmsAbvGrd','Fireplaces','GarageCars','GarageArea','SalePrice']].corr()

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
#objectのdescribe 一気に全部やると見ずらいから分けてやってみたわ
train[['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd']].describe(include = 'O')

In [ ]:
train[['MasVnrType', 'ExterQual', 'ExterCond', 'Foundation','BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2','Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual']].describe(include = 'O')      
  

In [ ]:
train[['Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition']].describe(include = 'O')

In [ ]:
import scipy.stats as st
y = train['SalePrice']
plt.figure(1); plt.title('Johnson SU')
sns.distplot(y, kde=False, fit=st.johnsonsu)
plt.figure(2); plt.title('Normal')
sns.distplot(y, kde=False, fit=st.norm)
plt.figure(3); plt.title('Log Normal')
sns.distplot(y, kde=False, fit=st.lognorm)
#これを見るとlogをとってみると正規分布に従っているような感じをかもしだしてるわ

In [ ]:
#カテゴリデータと目的変数のデータの関係性を見ている
for c in qualitative:
    train[c] = train[c].astype('category')
    if train[c].isnull().any():
        train[c] = train[c].cat.add_categories(['MISSING'])
        train[c] = train[c].fillna('MISSING')

def boxplot(x, y, **kwargs):
    sns.boxplot(x=x, y=y)
    x=plt.xticks(rotation=90)
f = pd.melt(train, id_vars=['SalePrice'], value_vars=qualitative)
g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False, size=5)
g = g.map(boxplot, "value", "SalePrice")
'''
箱ひげ図を見ることによって目的変数に影響を与えそうな特徴量はどれか見れそー
ExterQualなんか見ると各変数のsalepriceの分布がばらばらだからよさげ
neghtborhoodもいいっぽいけど変数多すぎてみる気にもならないけど一応あとでみとこ
'''

In [ ]:
df = pd.concat([train["SalePrice"] , train["ExterQual"].map({"Ex" : 4 , "Gd" : 3 , "TA":2,"Fa":1})] , axis = 1)

In [ ]:
df["ExterQual"] = df["ExterQual"].astype(int)

In [ ]:
df.corr()

In [ ]:
def anva(frame):
    anv = pd.DataFrame()
    anv["feature"] = qualitative #objectのやつ
    pvals = []
    for c in qualitative:
        samples = []
        for cls in frame[c].unique():
            s = frame[frame[c] == cls]["SalePrice"].values
            samples.append(s)
        pval = stats.f_oneway(*samples)[1]#一元配置分散分析を行っている
        pvals.append(pval)
    anv['pval'] = pvals
    return anv.sort_values('pval')

a = anva(train)
a["disparity"] = np.log(1./a['pval'].values)
plt.figure(figsize = (9,9))
sns.barplot(data = a ,x= 'feature' , y = "disparity")
x = plt.xticks(rotation=90)

'''
特徴量(neighborhood , street , exterqual等)の影響が小さいというのは各特徴量の変数の目的変数に対する平均は等しくなっているはず
→平均が一緒だと差が出てこないから影響与えてるかなんてわかんないよ的なやつだよねきっとうん。
これは一元配置分散分析によってそうならないとおかしいのかも？？？？？

'''

In [ ]:
a

In [ ]:
#特徴量をエンコーディングしている。
def encode(frame ,feature):#1つ1つの特徴量をエンコーディングする関数　frameにはtrain,featureには1つ1つの特徴量が入っている
    ordering = pd.DataFrame()
    ordering['val'] = frame[feature].unique()
    ordering.index = ordering.val
    ordering['spmean'] = frame[[feature , 'SalePrice']].groupby(feature).mean()["SalePrice"]#特徴量の変数ごとにsalepriceの平均を出している
    ordering = ordering.sort_values('spmean')#ソートした理由がマジでわかんないソートしなくてもできんだろとかおもうけど違うんか？あ？
    ordering['ordering'] = range(1,ordering.shape[0]+1)
    ordering = ordering["ordering"].to_dict()
    
    
    for cat ,o in ordering.items(): #trainに代入するって感じ
        frame.loc[frame[feature] == cat , feature+'_E'] = o
        
        
qual_encoded = []
for q in qualitative:
    encode(train , q)
    qual_encoded.append(q+'_E')
print(qual_encoded)

'''
質的変数は目的変数(SalePrice)の平均値に基づく順序に従ってエンコードしている。
'''

In [ ]:
#重要な特徴量を可視化してみてみる・

def spearman(frame , features):
    spr = pd.DataFrame()
    spr["feature"] = features
    spr["spearman"] = [frame[f].corr(frame["SalePrice"],"spearman") for f in features]
    spr = spr.sort_values('spearman')
    plt.figure(figsize = (6,0.25*len(features)))
    sns.barplot(data = spr , y = "feature" , x = "spearman" , orient = "h")
    
    
features = quantitative + qual_encoded
spearman(train , features)

In [ ]:
plt.figure(figsize= (16 , 16))
#saleprice と　特徴量との相関
plt.figure(1)
corr = train[quantitative + ["SalePrice"]].corr()
sns.heatmap(corr)


#saleprice と 
plt.figure(2)
corr = train[qual_encoded + ["SalePrice"]].corr()
sns.heatmap(corr)


plt.figure(3)
corr = pd.DataFrame(np.zeros([len(quantitative) + 1 , len(qual_encoded) +1]) , index = quantitative + ["SalePrice"] , columns = qual_encoded + ["SalePrice"])
for q1 in quantitative + ["SalePrice"]:
    for q2 in qual_encoded + ["SalePrice"]:
        corr.loc[q1 , q2] = train[q1].corr(train[q2])
sns.heatmap(corr)

In [ ]:
plt.figure(1)
corr = train[quantitative + ["SalePrice"]].corr()
sns.heatmap(corr)

In [ ]:
def pairplot(x, y , **kwargs):
    ax = plt.gca()
    ts = pd.DataFrame({"time" : x , "val" : y})
    ts = ts.groupby("time").mean()
    ts.plot(ax = ax)
    plt.xticks(rotation = 90)
    
f = pd.melt(train , id_vars = ["SalePrice"] , value_vars = quantitative + qual_encoded)
g = sns.FacetGrid(f , col = "variable" , col_wrap = 2 , sharex = False , sharey = False , size = 5)
g = g.map(pairplot , "value" , "SalePrice")

In [ ]:
train[["SalePrice" , "MoSold"]].corr()

In [ ]:
df = pd.concat([train["MoSold"] , (train["MoSold"] * train["MoSold"])] , axis = 1 )

In [ ]:
df["SalePrice"] = train["SalePrice"]

In [ ]:
df.corr()

In [ ]:
train["SalePrice"].describe()

In [ ]:
features = quantitative

standard = train[train["SalePrice"] < 200000]#平均よりも小さいものが入っている
pricey = train[train["SalePrice"] >= 200000]#平均よりも値が多きものが入っている。

diff = pd.DataFrame()
diff["feature"] = features
diff["difference"] = [(pricey[f].fillna(0.).mean() - standard[f].fillna(0.).mean())/(standard[f].fillna(0.).mean()) for f in features]
#価格が高いものと価格が低いものとの特徴量の変化のちがいを表している↑

sns.barplot(data = diff , x = "feature" , y = "difference")
x = plt.xticks(rotation = 90)

In [ ]:
features = quantitative + qual_encoded
model = TSNE(n_components = 2 , random_state = 0 , perplexity = 50)
X = train[features].fillna(0.).values
tsne = model.fit_transform(X)

std = StandardScaler()
s = std.fit_transform(X)
pca = PCA(n_components = 30)
pca.fit(s)
pc = pca.transform(s)
kmeans = KMeans(n_clusters = 5)
kmeans.fit(pc)

fr = pd.DataFrame({"tsne1":tsne[:,0] , "tsne2" : tsne[: , 1] , "cluster" : kmeans.labels_})
sns.lmplot(data = fr , x= "tsne1" , y = "tsne2" , hue = "cluster" , fit_reg = False)
print(np.sum(pca.explained_variance_ratio_))

In [ ]:
tsne.shape

In [ ]:
y = train["SalePrice"].values
def johnson(y):
    gamma , eta , epsilon , lbda = stats.johnsonsu.fit(y)
    yt = gamma + eta*np.arcsinh((y - epsilon) / lbda)
    return yt ,gamma , eta , epsilon ,lbda

def johnson_inverse(y , gamma , eta , epsilon , lbda ):
    return lbda*np.sinh((y - gamma) / eta) + epsilon


yt , g , et , ep , l = johnson(y)
yt2 = johnson_inverse(yt , g , et , ep , l)
plt.figure(1)
sns.distplot(yt)
plt.figure(2)
sns.distplot(yt2)

# ここから始めるるるるるる！！


In [ ]:
import pandas as pd
%matplotlib inline
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import KFold
from IPython.display import HTML, display
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns
train = pd.read_csv("/kaggle/input/home-data-for-ml-course/train.csv")
test = pd.read_csv("/kaggle/input/home-data-for-ml-course/test.csv")
sample_submission = pd.read_csv("/kaggle/input/home-data-for-ml-course/sample_submission.csv")
print(train.shape)
print(test.shape)

In [ ]:
train.drop("Id" , axis = 1)
test.drop("Id" , axis = 1)
train["SalePrice"] = np.log1p(train["SalePrice"])
train = train.drop(train[(train["GrLivArea"] > 4000) & (train["SalePrice"] < 300000)].index)
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.SalePrice.values
all_data = pd.concat((train , test)).reset_index(drop = True)
all_data.drop(["SalePrice"] , axis = 1 , inplace = True)

In [ ]:
print(all_data.shape)

In [ ]:
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending = False)[:30]
missing_data = pd.DataFrame({'Missing Ratio':all_data_na})

missing_data.head(20)

In [ ]:
import seaborn as sns
f , ax = plt.subplots(figsize = (15,12))
plt.xticks(rotation = 90)
sns.barplot(all_data_na.index , all_data_na)

In [ ]:
corrmat = train.corr()

In [ ]:
plt.subplots(figsize = (12,9))
sns.heatmap(corrmat , vmax = 0.9 , square = True)

In [ ]:
all_data["PoolQC"] = all_data["PoolQC"].fillna("None")

In [ ]:
k = ["MiscFeature" , "Alley" , "Fence" , "FireplaceQu"] 
for i in k:
    all_data[i] = all_data[i].fillna("None")

In [ ]:
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(lambda x : x.fillna(x.median()))

In [ ]:
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')

In [ ]:
#欠損値を穴埋めする！！！！
all_data['GarageCars'] = all_data['GarageCars'].fillna(2)
all_data["GarageYrBlt"] = all_data["GarageYrBlt"].fillna(0)
all_data['GarageArea'] = all_data['GarageArea'].fillna(0)
all_data['BsmtFinSF1'] = all_data['BsmtFinSF1'].fillna(0)
all_data['BsmtFinSF2'] = all_data['BsmtFinSF2'].fillna(0)
all_data['BsmtUnfSF'] = all_data[ 'BsmtUnfSF'].fillna(0)
all_data['TotalBsmtSF'] = all_data['TotalBsmtSF'].fillna(988)
all_data['BsmtFullBath'] = all_data['BsmtFullBath'].fillna(0)
all_data['BsmtHalfBath'] = all_data['BsmtHalfBath'].fillna(0)
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])
all_data = all_data.drop(['Utilities'], axis=1)
all_data["Functional"] = all_data["Functional"].fillna("Typ")
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])
all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")

In [ ]:
#欠損値がないことを確認する
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head()

In [ ]:
#もう少し特徴量エンジニアリング
#MSSubClass=The building class
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)


#Changing OverallCond into a categorical variable
all_data['OverallCond'] = all_data['OverallCond'].astype(str)


#Year and month sold are transformed into categorical features.
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

In [ ]:
# labelencodeで行っている！！

#順序のカテゴリを含んでいるものを数値にエンコーディングしている(ただ単にラベルエンコーディングを行っている)
from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')


#ターゲットエンコーディングで行ってみる

# process columns, apply LabelEncoder to categorical features
for c in cols:
    lbl = LabelEncoder()
    lbl.fit(list(all_data[c].values)) 
    all_data[c] = lbl.transform(list(all_data[c].values))

# shape        
print('Shape all_data: {}'.format(all_data.shape))

#ターゲットエンコーディングを行ってみる
train = all_data[:ntrain]
test = all_data[ntrain:]

cols1 = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')
cols2 = ['FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold']
import category_encoders as ce
target_enc = ce.TargetEncoder(cols=cols1)
target_enc.fit(train[cols2], y_train)

train[cols2] = target_enc.transform(train[cols2])
test[cols2] = target_enc.transform(test[cols2])



all_data = pd.concat((train , test)).reset_index(drop = True)

In [ ]:
#面積のデータは住宅価格に大きく影響を与える
#そのため各住宅の地下室、1階、2階の面積の合計面積の新しい特徴量を作成する
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

In [ ]:
#scipy.statsをimportして、skew()で歪度を算出できます。
from scipy import stats
from scipy.stats import norm, skew

#オブジェクトじゃないカラム名を算出している
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)

In [ ]:
# Box Cox transformを使用した変換
skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    #all_data[feat] += 1
    all_data[feat] = boxcox1p(all_data[feat], lam)
    
#all_data[skewed_features] = np.log1p(all_data[skewed_features])

In [ ]:
all_data = pd.get_dummies(all_data)
print(all_data.shape)

In [ ]:
#テストデータとトレーニングデータを元に戻している
train = all_data[:ntrain]
test = all_data[ntrain:]

In [ ]:
y_train.shape

In [ ]:
train.shape

# こっから予測を始める

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import LinearRegression , RidgeCV , LassoCV , ElasticNetCV

In [ ]:
#クロスバリデーションを使用する
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [ ]:
#lasso
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

#Elastic Net Regression
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

#Kernel Ridge Regression
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

#Gradient Boosting Regression 
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

#XGBoost
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

#LightGBM
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

#予測を行う！！
average_df = pd.DataFrame()

lasso.fit(train, y_train)
lasso_pred = np.exp(lasso.predict(test))
average_df["lasso"] = lasso_pred

ENet.fit(train, y_train)
ENet_pred = np.exp(ENet.predict(test))
average_df["ENet"] = ENet_pred

KRR.fit(train, y_train)
KRR_pred = np.exp(KRR.predict(test))
average_df["KRR"] = KRR_pred

GBoost.fit(train, y_train)
GBoost_pred = np.exp(GBoost.predict(test))
average_df["GBoost"] = GBoost_pred

model_xgb.fit(train, y_train)
model_xgb_pred = np.exp(model_xgb.predict(test))
average_df["model_xgb"] = model_xgb_pred

model_lgb.fit(train, y_train)
model_lgb_pred = np.exp(model_lgb.predict(test))
average_df["model_lgb"] = model_lgb_pred


In [ ]:
#average_df["Ave"] = average_df.mean(axis=1) 
#average_df

In [ ]:
#提出データにする
#sample_submission["SalePrice"] = average_df["Ave"]
#sample_submission.to_csv('hose_price21.csv',index=False)

In [ ]:
class AveragingModels(BaseEstimator , RegressorMixin , TransformerMixin):
    
    def __init__(self ,models):
        self.models = models
        
    def fit(self,X,y):
        self.models_ = [clone(x) for x in self.models]
        
        
        for model in self.models_:
            model.fit(X,y)
            
        return self
    
    
    def predict(self , X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        
        return np.mean(predictions , axis =1)

# ここから予測を始める

In [ ]:
#特徴量を削減する
columns1 = ['TotalBsmtSF','1stFlrSF','2ndFlrSF',"LandSlope","Street","Functional" ,"Alley","SaleType_COD", 'SaleCondition_Partial',
            "RoofStyle_Shed","RoofMatl_WdShngl","Neighborhood_Veenker","MiscFeature_TenC",
           "MasVnrType_Stone","MSZoning_RM","LotConfig_Inside","LandContour_Lvl","HouseStyle_SLvl","Heating_Wall","GarageType_None"
           ,"Foundation_Wood" , "Exterior2nd_VinylSd" , "Exterior1st_WdShing","Electrical_SBrkr","Condition2_RRNn","Condition1_RRNn"
           ,"BldgType_TwnhsE","BsmtFinSF2"]
combine = [train , test]

for data in combine:
    for i in columns1:
        data.drop(i , axis = 1 , inplace = True)

In [ ]:
print(train.shape)
print(test.shape)
print(train_df.shape)
print(test_df.shape)


train_df = train
test_df = test

In [ ]:
average_df = pd.DataFrame()
predict_df = pd.DataFrame()


lasso.fit(train, y_train)
lasso_pred = np.exp(lasso.predict(test))
average_df["lasso"] = lasso_pred
lasso_pred2 = np.exp(lasso.predict(train))
predict_df["lasso"] = lasso_pred2

ENet.fit(train, y_train)
ENet_pred = np.exp(ENet.predict(test))
average_df["ENet"] = ENet_pred
ENet_pred2 = np.exp(ENet.predict(train))
predict_df["ENet"] = ENet_pred2


KRR.fit(train, y_train)
KRR_pred = np.exp(KRR.predict(test))
average_df["KRR"] = KRR_pred
KRR_pred2 = np.exp(KRR.predict(train))
predict_df["KRR"] = KRR_pred2

GBoost.fit(train, y_train)
GBoost_pred = np.exp(GBoost.predict(test))
average_df["GBoost"] = GBoost_pred
GBoost_pred2 = np.exp(GBoost.predict(train))
predict_df["GBoost"] = GBoost_pred2

model_xgb.fit(train, y_train)
model_xgb_pred = np.exp(model_xgb.predict(test))
average_df["model_xgb"] = model_xgb_pred
model_xgb_pred2 = np.exp(model_xgb.predict(train))
predict_df["model_xgb"] = model_xgb_pred2

model_lgb.fit(train, y_train)
model_lgb_pred = np.exp(model_lgb.predict(test))
average_df["model_lgb"] = model_lgb_pred
model_lgb_pred2 = np.exp(model_lgb.predict(train))
predict_df["model_lgb"] = model_lgb_pred2

#平均を求める

average_df["Ave"] = average_df.mean(axis=1) 
predict_df["Ave"] = predict_df.mean(axis=1) 

average_df

In [ ]:
#予測の誤差がどのくらいなのかを計算する
def error(actual , predicted):
    actual = np.log(actual)
    predicted = np.log(predicted)
    return np.sqrt(np.sum(np.square(actual - predicted))/len(actual))


print("error: {}".format(error(predict_df["Ave"].values , y_train )))

In [ ]:
pd.options.display.max_columns = None


In [ ]:
train.corr().sort_values(by = "BsmtFinSF2")

In [ ]:
#特徴量を削減した後の誤差を求める

#削除したい特徴量
columns2 = ["GarageCars"]

train_df = train.copy()
test_df = test.copy()

combine2 = [train_df , test_df]



for data in combine2:
    for i in columns2:
        data.drop(i , axis = 1 , inplace = True)

average_df2 = pd.DataFrame()
predict_df2 = pd.DataFrame()


lasso.fit(train_df, y_train)
lasso_pred = np.exp(lasso.predict(test_df))
average_df2["lasso"] = lasso_pred
lasso_pred2 = np.exp(lasso.predict(train_df))
predict_df2["lasso"] = lasso_pred2

ENet.fit(train_df, y_train)
ENet_pred = np.exp(ENet.predict(test_df))
average_df2["ENet"] = ENet_pred
ENet_pred2 = np.exp(ENet.predict(train_df))
predict_df2["ENet"] = ENet_pred2


KRR.fit(train_df, y_train)
KRR_pred = np.exp(KRR.predict(test_df))
average_df2["KRR"] = KRR_pred
KRR_pred2 = np.exp(KRR.predict(train_df))
predict_df2["KRR"] = KRR_pred2

GBoost.fit(train_df, y_train)
GBoost_pred = np.exp(GBoost.predict(test_df))
average_df2["GBoost"] = GBoost_pred
GBoost_pred2 = np.exp(GBoost.predict(train_df))
predict_df2["GBoost"] = GBoost_pred2

model_xgb.fit(train_df, y_train)
model_xgb_pred = np.exp(model_xgb.predict(test_df))
average_df2["model_xgb"] = model_xgb_pred
model_xgb_pred2 = np.exp(model_xgb.predict(train_df))
predict_df2["model_xgb"] = model_xgb_pred2

model_lgb.fit(train_df, y_train)
model_lgb_pred = np.exp(model_lgb.predict(test_df))
average_df2["model_lgb"] = model_lgb_pred
model_lgb_pred2 = np.exp(model_lgb.predict(train_df))
predict_df2["model_lgb"] = model_lgb_pred2

#平均を求める
average_df2["Ave"] = average_df.mean(axis=1) 
predict_df2["Ave"] = predict_df.mean(axis=1)
average_df2

In [ ]:
print("error2: {}".format(error(predict_df2["Ave"].values , y_train )))

In [ ]:
#提出用のデータ作成

#sample_submission["SalePrice"] = average_df2["Ave"]
#sample_submission.to_csv('house_price30.csv',index=False)

In [ ]:
,"GarageCars"

In [ ]:
train[]

In [ ]:
train_df["BsmtFinSF2"]

In [ ]:
print(train.shape)
print(test.shape)
print(train_df.shape)
print(test_df.shape)